# Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

Objetivo del Ejercicio: Utilizando un conjunto de datos que hemos compartido, se busca evaluar si existen diferencias significativas en el número de vuelos reservados según el nivel educativo de los clientes. Para ello, los pasos que deberas seguir son:

1. Preparación de Datos: 

    - Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

2. Análisis Descriptivo: 

    - Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

3. Prueba Estadística:

    - Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

### Importamos las librerías que necesitamos

In [31]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
from src import support as sp

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Creación de muestras 
# -----------------------------------------------------------------------
from sklearn.model_selection import train_test_split

In [32]:
# Cargamos los datos generados en la Fase 1
df_combined = pd.read_csv('../data/df_combined.csv', index_col=0)

In [33]:
# Mostramos las primeras filas del DataFrame
df_combined.head(3) 

loyalty_number  year  month  flights_booked  flights_with_companions  \
0          100018  2017      1               3                        0   
1          100102  2017      1              10                        4   
2          100140  2017      1               6                        0   

   total_flights  distance  points_accumulated  points_redeemed  \
0              3      1521               152.0                0   
1             14      2030               203.0                0   
2              6      1200               120.0                0   

   dollar_cost_points_redeemed          province          city postal_code  \
0                            0           Alberta      Edmonton     T9G 1W3   
1                            0           Ontario       Toronto     M1R 4K3   
2                            0  British Columbia  Dawson Creek     U5I 4F1   

   gender education   salary marital_status loyalty_card      clv  \
0  Female  Bachelor  92552.0        Married       Aurora  7919.20   
1    Male   College      NaN         Single         Nova  2887.74   
2  Female   College      NaN       Divorced         Nova  2838.07   

  enrollment_type  enrollment_year  enrollment_month  cancellation_year  \
0        Standard             2016                 8                NaN   
1        Standard             2013                 3                NaN   
2        Standard             2016                 7                NaN   

   cancellation_month  
0                 NaN  
1                 NaN  
2                 NaN

- Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

In [41]:
# Filtrar el conjunto de datos para incluir las columnas 'flights_booked' y 'education'
df_filtered = df_combined[['flights_booked', 'education']]
df_filtered.shape

(401688, 2)

* Si solo filtramos por esas 2 columnas, luego perdemos la referencia de quien son estos clientes, por lo cuál, filtraremos por 3 columnas

In [42]:
# Filtrar el conjunto de datos para incluir las columnas 'loyalty_number', 'flights_booked' y 'education'
df_filtered = df_combined[['loyalty_number', 'flights_booked', 'education']]
df_filtered.shape

(401688, 3)

- Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

In [43]:
df_filtered.groupby('education')['flights_booked'].describe()

count      mean       std  min  25%  50%  75%   max
education                                                                   
Bachelor              251400.0  4.090354  5.214194  0.0  0.0  1.0  8.0  21.0
College               101712.0  4.145656  5.237434  0.0  0.0  1.0  8.0  21.0
Doctor                 17616.0  4.149296  5.246376  0.0  0.0  1.0  8.0  21.0
High School or Below   18768.0  4.142210  5.229254  0.0  0.0  1.0  8.0  21.0
Master                 12192.0  4.167815  5.204100  0.0  0.0  1.0  8.0  21.0

- Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

Lo primero que debemos hacer en un A/B testing es verificar que los datos son coherentes y están listos. En este caso, ya tenemos este paso concluído.

Dado que cada cliente tiene 24 entradas (una por cada mes en dos años), vamos a calcular el promedio mensual de vuelos reservados por cada cliente. Así tendremos una sola fila por cliente con la media de vuelos reservados.

In [44]:
# Agrupamos por loyalty_number y education y calculamos la media de vuelos reservados por cada cliente
df_filtered_ab = df_filtered.groupby(['loyalty_number', 'education'])['flights_booked'].mean().reset_index() 
df_filtered_ab.tail()

loyalty_number education  flights_booked
16732          999902   College        7.416667
16733          999911    Doctor        0.000000
16734          999940  Bachelor        2.625000
16735          999982   College        0.916667
16736          999986  Bachelor        5.041667

### Ahora vamos a crear nuestro test_group, categorizando la columna education en 2 grupos: 
    * GRUPO A: Clientes con nivel de educación inferior: High School or Below y College
    * GRUPO B: Clientes con nivel de educación superior: Bachelor, Doctor y Master

El orden de los niveles de educación de menor a mayor es:

1. **High School or Below** (Secundaria o menos)
2. **College** (Universidad, generalmente implica que no se completó un título universitario)
3. **Bachelor** (Licenciatura)
4. **Master** (Máster)
5. **Doctor** (Doctorado)

In [46]:
df_filtered_ab.education.value_counts() # Miramos como están distribuidos los niveles de educación

education
Bachelor                10475
College                  4238
High School or Below      782
Doctor                    734
Master                    508
Name: count, dtype: int64

In [45]:
# Usamos el apply para aplicar nuestra función que categoriza los distintos niveles de education
group_a = ['High School or Below', 'College']
group_b = ['Bachelor', 'Master', 'Doctor']

df_filtered_ab['test_group'] = df_filtered_ab['education'].apply(lambda x: sp.categorize(x, group_a, group_b))

In [48]:
df_filtered_ab.sample(5)

loyalty_number             education  flights_booked test_group
8629          564813              Bachelor        0.916667    group_b
5432          393975              Bachelor        0.000000    group_b
2483          235220  High School or Below        0.000000    group_a
6763          464447               College        5.375000    group_a
4266          330278                Doctor        4.083333    group_b

### Planteamos las hipótesis

- Hipótesis nula (H0): No existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.
- Hipótesis alternativa (H1): Existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.

### Definimos si es un problema de medias o de proporciones

In [50]:
pd.crosstab(df_filtered_ab.flights_booked, df_filtered_ab.test_group).shape # Con una crosstab confirmamos que es un problema de médias, nuestra crosstab tiene 225 filas.

(225, 2)

In [51]:
sp.determine_problem_type(df_filtered_ab, 'flights_booked') # Tenemos una función que también nos hace esta comprobación 

'means'

### Aplicamos una prueba de normalidad en nuestra métrica

In [52]:
sp.normality_test(df_filtered_ab, 'flights_booked')

The data for the metric 'flights_booked' does not follow a normal distribution (p-value = 0.0000).


/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


(0.9311605095863342, 0.0)

* Vemos que no podemos tener el p-valor en cuenta debido al alto número del conjunto de datos. 
* Para solucionar esto y mejorar la precisión de nuestro análisis, vamos a separar muestras estratificadas y mas pequeñas del conjunto de datos.

In [53]:
# Vamos a dropar la columna de categorias que habíamos creado anteriormente
df_filtered_ab.drop(columns=['test_group'], inplace=True)
df_filtered_ab.head(2)

loyalty_number education  flights_booked
0          100018  Bachelor        6.541667
1          100102   College        7.208333

In [54]:
# Definimos el tamaño de la muestra deseada
sample_size = 1000  # Ajusta el tamaño según sea necesario

# Obtenemos las categorías únicas
education_categories = df_filtered_ab.education.unique()

# Creamos una muestra equilibrada para cada categoría
sampled_dfs = [] # Creamos una lista que guardará los dfs con muestras de cada categoria
for category in education_categories:
    df_category = df_filtered_ab[df_filtered_ab['education'] == category] # Creamos un df con una categoria de educación
    sample_size_category = min(len(df_category), sample_size // len(education_categories)) # La función min compara dos valores y devuelve el menor. Esto garantiza que no intentemos tomar más muestras de las que están disponibles en la categoría
    sampled_df_category = df_category.sample(n=sample_size_category, random_state=42) # sample es un método de la librería train_test_split para crear muestras, le pasamos el número de muestras calculado previamente
    sampled_dfs.append(sampled_df_category)

# Concatenamos todas las muestras
df_sampled = pd.concat(sampled_dfs)
df_sampled.education.value_counts()

education
Bachelor                200
College                 200
Master                  200
High School or Below    200
Doctor                  200
Name: count, dtype: int64

In [55]:
df_sampled.head()

loyalty_number education  flights_booked
8256           544690  Bachelor        4.666667
16373          979330  Bachelor        1.500000
14552          880619  Bachelor        2.000000
6914           472674  Bachelor        6.625000
5000           370090  Bachelor        5.416667

In [56]:
# Volvemos a crear nuestros grupos ahora que tenemos una muestra mas pequeña
group_a = ['High School or Below', 'College']
group_b = ['Bachelor', 'Master', 'Doctor']

df_sampled['test_group'] = df_sampled['education'].apply(lambda x: sp.categorize(x, group_a, group_b))

In [57]:
df_sampled.sample(5)

loyalty_number             education  flights_booked test_group
9163           592251              Bachelor        5.083333    group_b
9781           624463  High School or Below        5.083333    group_a
13088          800800               College        4.750000    group_a
15774          948325              Bachelor        3.458333    group_b
5465           395871                Master        4.625000    group_b

In [58]:
df_sampled.shape

(1000, 4)

In [59]:
# Volvemos a aplicar la prueba de normalidad en el df_sample
sp.normality_test(df_sampled, 'flights_booked')

The data for the metric 'flights_booked' does not follow a normal distribution (p-value = 0.0000).


(0.9289609789848328, 2.3417810297770827e-21)

Hemos obtenido un p-valor distinto al anterior, pero igualmente concluimos que los datos no siguen una distribución normal, por lo tanto usaremos una prueba no paramétrica

In [60]:
sp.mann_whitney_test(df_sampled, ['flights_booked'], 'group_a', 'group_b')

For the metric 'flights_booked', the medians are the same (p-value = 0.7465).


### Interpretación de Resultados de la Prueba de Mann-Whitney U

**Métrica Evaluada:** `flights_booked`

**Resultado de la Prueba:**
- **p-value:** 0.7465

**Hipótesis:**
- **Hipótesis nula (H0):** No existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.
- **Hipótesis alternativa (H1):** Existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.

**Interpretación:**

El resultado de la prueba de Mann-Whitney U indica un p-value de 0.7465. Dado que este valor es considerablemente mayor que el nivel de significancia comúnmente aceptado de 0.05, no rechazamos la hipótesis nula.

**Conclusión:**

Con un p-value de 0.7465, no encontramos evidencia suficiente para afirmar que existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B. Por lo tanto, según los datos analizados, no hay una diferencia significativa en el número de vuelos reservados entre estos grupos. Esto sugiere que el nivel educativo no tiene un impacto significativo en la cantidad de vuelos reservados por los clientes en los grupos comparados.

---

Podríamos seguir profundizando este análisis y realizando otros A/B testing cambiando las categorias educativas que pertenecen a los grupos A y B, o incluso podríamos realizar pruebas con otras métricas distintas. 